
### This version does not use dask to load imagery


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')
#comment test, 15 Sep 2023

#Steve comment

I have not tried to untangle most of the code below, just focusing on the random forest part. That said, my approach with what seems like a similar project is to separate out labeled data from unlabeled data. Then just load and use labeled data to do model training and testing.

It appears you have roughly 105M samples/rows of total pixels, each at 140 bands. Huge. My guess you have actually only labeled a fraction of that, say 100K. It seems you should only be working with that 100K in this notebook. Maybe that is all you are doing and I have not deciphered the code below well enough. But my take is that 100K samples with 140 features should likely run directly in Colab, probably with the Pro+ version. I'd be happy to try it :)

Another comment I leave below is that I only preprocess my data once and then store the resulting numpy matrix out to file. I then can just load this file subsequently, skipping all the preprocessing steps. My numpy matrix is going to be about the same size as yours and I can tell you it is relatively fast to load it. Again, maybe you are doing this and I haven't untangled the code to see it.

I did not see any testing going on. It appears you rely totally on oob scoring and the training set for evaluation. If so, I believe you might want to rethink this. Standard practice is to show results from a test set. Much more reliable for what will happen in wild.

Finally, I urge you to consider a more modern modeling approach based on boosting. It is kind of the next generation of tree ensembles. It should not cause you to change much and could have many benefits.

In [ ]:
from osgeo import gdal, ogr, gdal_array # I/O image data
import numpy as np # math and array handling
import matplotlib.pyplot as plt # plot figures
from sklearn.ensemble import RandomForestClassifier # classifier
import pandas as pd # handling large data as table sheets
from sklearn.metrics import classification_report, accuracy_score,confusion_matrix  # calculating measures for accuracy assessment

import seaborn as sn

import datetime

import joblib

In [ ]:
import pickle

In [ ]:
import os

# print(os.environ['GDAL_NUM_THREADS'])
os.environ['PROJ_LIB'] = "/work/pi_gstuart_umass_edu/kate/conda/share/proj"

In [ ]:
# Tell GDAL to throw Python exceptions, and register all drivers
gdal.UseExceptions()
gdal.AllRegister()

# define a number of trees that should be used (default = 500)
est = 500

# how many cores should be used?
# -1 -> all available cores
n_cores = 60

# the remote sensing image you want to classify
img_RS = r'work/pi_cschweik_umass_edu/Ryan/Modeling_stack/Mid_High_Stacked_Clipped.tif'

# training and validation as shape files
training = r'/work/pi_gstuart_umass_edu/kate/OTH_Training_Data/OTH_Polygons_v3_relassed.shp'
validation = r'/work/pi_gstuart_umass_edu/kate/OTH_Training_Data/OTH_Polygons_v3_relassed.shp'

# what is the attributes name of your classes in the shape file (field name of the classes)?
attribute = 'ReClaSch A'


# directory, where the classification image should be saved:
classification_image = r'/work/pi_gstuart_umass_edu/kate/Classification_products/OTH_all_reclass_classification_v1.gtif'

# directory, where the all meta results should be saved:
results_txt = r'/work/pi_gstuart_umass_edu/kate/Classification_products/OTH_all_reclass_classification_v1.txt'

# laod training data and show all shape attributes

#model_dataset = gdal.Open(model_raster_fname)
shape_dataset = ogr.Open(training)
shape_layer = shape_dataset.GetLayer()

# extract the names of all attributes (fieldnames) in the shape file
attributes = []
ldefn = shape_layer.GetLayerDefn()
for n in range(ldefn.GetFieldCount()):
    fdefn = ldefn.GetFieldDefn(n)
    attributes.append(fdefn.name)

# print the attributes
print('Available attributes in the shape file are: {}'.format(attributes))

# prepare results text file:

print('Random Forest Classification', file=open(results_txt, "a"))
print('Processing: {}'.format(datetime.datetime.now()), file=open(results_txt, "a"))
print('-------------------------------------------------', file=open(results_txt, "a"))
print('PATHS:', file=open(results_txt, "a"))
print('Image: {}'.format(img_RS), file=open(results_txt, "a"))
print('Training shape: {}'.format(training) , file=open(results_txt, "a"))
print('Vaildation shape: {}'.format(validation) , file=open(results_txt, "a"))
print('      choosen attribute: {}'.format(attribute) , file=open(results_txt, "a"))
print('Classification image: {}'.format(classification_image) , file=open(results_txt, "a"))
print('Report text file: {}'.format(results_txt) , file=open(results_txt, "a"))
print('-------------------------------------------------', file=open(results_txt, "a"))

Available attributes in the shape file are: ['Transect', 'PointNum', 'SubClass', 'Northing', 'Easting', 'Altitude', 'Notes', 'Class', 'Raw Subcla', 'Pre Angle', 'Post Angle', 'mu', 'Class mu', 'Pre/Post', 'Hydro', 'ReClaSch A']


#Steve comment

If below takes a long time, is there no way to cache it? Compute it once but then store results (looks like numpy array) out to file. Then just load that file in in place of this code? I do that with my project and gives me huge time savings after you bite the bullet once.

In [ ]:
# load image data
######THIS IS STEP THAT TAKES A LONG TIME#######

img_ds = gdal.Open(img_RS, gdal.GA_ReadOnly)

img = np.zeros((img_ds.RasterYSize, img_ds.RasterXSize, img_ds.RasterCount),
               gdal_array.GDALTypeCodeToNumericTypeCode(img_ds.GetRasterBand(1).DataType))
print(img.shape, gdal_array.GDALTypeCodeToNumericTypeCode(img_ds.GetRasterBand(1).DataType), img.itemsize)
print(img_ds.GetRasterBand(1).ReadAsArray().itemsize)
for b in range(img.shape[2]):
    print(b)
    img[:, :, b] = img_ds.GetRasterBand(b + 1).ReadAsArray()

row = img_ds.RasterYSize
col = img_ds.RasterXSize
band_number = img_ds.RasterCount

print('Image extent: {} x {} (row x col)'.format(row, col))
print('Number of Bands: {}'.format(band_number))


print('Image extent: {} x {} (row x col)'.format(row, col), file=open(results_txt, "a"))
print('Number of Bands: {}'.format(band_number), file=open(results_txt, "a"))
print('---------------------------------------', file=open(results_txt, "a"))
print('TRAINING', file=open(results_txt, "a"))
print('Number of Trees: {}'.format(est), file=open(results_txt, "a"))

(12434, 13246, 142) <class 'numpy.float32'> 4
4
0
1


#Steve comment

It appears to me we will end up with 12kx13k samples/rows (156M) with 142 bands each. That is huge and will impact memory significantly. News you already know :)

However, I assume only a small fraction is actually labeled and suitable for training. I am going to guess 100K samples have been labeled.

I am going to suggest a couple things down below to (a) deal with memory and (b) improve the model.

In [ ]:
# laod training data from shape file
# img_ds = gdal.Open(img_RS, gdal.GA_ReadOnly)

#model_dataset = gdal.Open(model_raster_fname)
shape_dataset = ogr.Open(training)
shape_layer = shape_dataset.GetLayer()

mem_drv = gdal.GetDriverByName('MEM')
mem_raster = mem_drv.Create('',img_ds.RasterXSize,img_ds.RasterYSize,1,gdal.GDT_UInt16)
#mem_raster.SetProjection(img_ds.GetProjection())
mem_raster.SetGeoTransform(img_ds.GetGeoTransform())
mem_band = mem_raster.GetRasterBand(1)
mem_band.Fill(0)
mem_band.SetNoDataValue(0)

att_ = 'ATTRIBUTE='+attribute
# http://gdal.org/gdal__alg_8h.html#adfe5e5d287d6c184aab03acbfa567cb1
# http://gis.stackexchange.com/questions/31568/gdal-rasterizelayer-doesnt-burn-all-polygons-to-raster
err = gdal.RasterizeLayer(mem_raster, [1], shape_layer, None, None, [1],  [att_,"ALL_TOUCHED=TRUE"])
assert err == gdal.CE_None

roi = mem_raster.ReadAsArray()

In [ ]:
# # Display images
# plt.subplot(121)
# plt.imshow(img[:, :, 0], cmap=plt.cm.Greys_r)
# plt.title('RS image - first band')

# plt.subplot(122)
# plt.imshow(roi, cmap=plt.cm.Spectral)
# plt.title('Training Image')

# plt.show()

# Number of training pixels:
n_samples = (roi > 0).sum()
print('{n} training samples'.format(n=n_samples))
print('{n} training samples'.format(n=n_samples), file=open(results_txt, "a"))

# What are our classification labels?
labels = np.unique(roi[roi > 0])
print('training data include {n} classes: {classes}'.format(n=labels.size, classes=labels))
print('training data include {n} classes: {classes}'.format(n=labels.size, classes=labels), file=open(results_txt, "a"))

# Subset the image dataset with the training image = X
# Mask the classes on the training dataset = y
# These will have n_samples rows
X = img[roi > 0, :]
y = roi[roi > 0]

print('Our X matrix is sized: {sz}'.format(sz=X.shape))
print('Our y array is sized: {sz}'.format(sz=y.shape))

#Steve comment

I'm wondering if your classifier a bit underspecified. And wonder if we can make it more efficient (less of a time and memory hog). Here is a proposed change:
<pre>
rf = RandomForestClassifier(
    n_estimators=100,      # Start lower, let warm_start add more
    max_depth=None,        # Can let trees grow fully
    min_samples_leaf=1,
    max_features='sqrt',   # sqrt(140) ≈ 12 features per split
    oob_score=True,
    warm_start=True,       # finding optimal n_estimators
    n_jobs=-1,
    verbose=1,
    random_state=42        # for reproducibility
)
</pre>

###train-test split

I did not see it but assume at some point you will split your labeled data into a training set and a test set.
<pre>
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,
    random_state=42  #for reproducibility
)

#Training set: 80K rows.
#Test set: 20K rows.

# Train incrementally - potential memory savings
step = 100  # Add 100 trees each time
max_total_estimators = 1000
plateaued = 0
best_n_estimators = None

# First fit
rf.fit(X_train, y_train)
current_estimators = rf.n_estimators_
min_oob_score = rf.oob_score_

while current_estimators < max_total_estimators:
    # Add 100 more trees
    current_estimators += step
    rf.set_params(n_estimators=current_estimators)
    rf.fit(X_train, y_train)  #adds whatever the difference is between the current n_estimators and what's already there.
    
    # Check if OOB score improved
    if rf.oob_score_ > min_oob_score + 0.0001:
        min_oob_score = rf.oob_score_
        best_n_estimators = rf.n_estimators_  # Track the best point
        plateaued = 0
    else:
        plateaued += 1
    
    if plateaued == 3:  # No improvement for 3 iterations
        break

# Roll back to best number of estimators
if best_n_estimators:
    rf.set_params(n_estimators=best_n_estimators)
    rf.fit(X_train, y_train)  # One final fit to get back to best state

# For memory efficiency, evaluate in chunks:

from sklearn.metrics import precision_recall_fscore_support, accuracy_score, confusion_matrix

def evaluate_in_chunks(model, X, y, chunk_size=1000000):
    predictions = []
    n_samples = len(X)
    
    # Get predictions in chunks
    for i in range(0, n_samples, chunk_size):
        chunk_end = min(i + chunk_size, n_samples)
        X_chunk = X[i:chunk_end]
        chunk_preds = model.predict(X_chunk)
        predictions.extend(chunk_preds)
    
    predictions = np.array(predictions)
    
    # Calculate all metrics
    precision, recall, f1, _ = precision_recall_fscore_support(y, predictions, average='weighted')
    
    metrics = {
        'accuracy': accuracy_score(y, predictions),
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'confusion_matrix': confusion_matrix(y, predictions)
    }
    
    return metrics

results = evaluate_in_chunks(rf, X_test, y_test)

print("Accuracy:", results['accuracy'])
print("Precision:", results['precision'])
print("Recall:", results['recall'])
print("F1 Score:", results['f1'])
print("\nConfusion Matrix:\n", results['confusion_matrix'])
</pre>

In [ ]:
rf = RandomForestClassifier(n_estimators=est, oob_score=True, verbose=1, n_jobs=n_cores)

# verbose = 2 -> prints out every tree progression
# rf = RandomForestClassifier(n_estimators=est, oob_score=True, verbose=2, n_jobs=n_cores)



X = np.nan_to_num(X)
rf2 = rf.fit(X, y)


# Save the trained model to a file
pickle_file = '/work/pi_gstuart_umass_edu/kate/Classification_products/RF_alldata_v1.pkl'
with open(pickle_file, 'wb') as file:
    pickle.dump(rf2, file)

print ('model pickled')

# With our Random Forest model fit, we can check out the "Out-of-Bag" (OOB) prediction score:

print('--------------------------------', file=open(results_txt, "a"))
print('TRAINING and RF Model Diagnostics:', file=open(results_txt, "a"))
print('OOB prediction of accuracy is: {oob}%'.format(oob=rf.oob_score_ * 100))
print('OOB prediction of accuracy is: {oob}%'.format(oob=rf.oob_score_ * 100), file=open(results_txt, "a"))


# we can show the band importance:
bands = range(1,img_ds.RasterCount+1)

for b, imp in zip(bands, rf2.feature_importances_):
    print('Band {b} importance: {imp}'.format(b=b, imp=imp))
    print('Band {b} importance: {imp}'.format(b=b, imp=imp), file=open(results_txt, "a"))


# Let's look at a crosstabulation to see the class confusion.
# To do so, we will import the Pandas library for some help:
# Setup a dataframe -- just like R
# Exception Handling because of possible Memory Error

try:
    df = pd.DataFrame()
    df['truth'] = y
    df['predict'] = rf.predict(X)

except MemoryError:
    print('Crosstab not available ')

else:
    # Cross-tabulate predictions
    print(pd.crosstab(df['truth'], df['predict'], margins=True))
    print(pd.crosstab(df['truth'], df['predict'], margins=True), file=open(results_txt, "a"))

# Predicting the rest of the image

# generate mask image from red band
mask = np.copy(img[:,:,0])
mask[mask > 0.0] = 1.0 # all actual pixels have a value of 1.0

# Take our full image and reshape into long 2d array (nrow * ncol, nband) for classification
old_shape = img.shape
new_shape = (img.shape[0] * img.shape[1], img.shape[2])
# img = img[:, :, :int(img.shape[2])].reshape(new_shape)
img = img.reshape(new_shape)

print('Reshaped from {o} to {n}'.format(o=old_shape, n=img.shape))

In [ ]:
# img = np.nan_to_num(img)
img[np.isnan(img)] = 0.0
# class_prediction = rf.predict(img)

print ("DONE!")


#Steve comment

My suggestion is to go with boosting over random forest. It is more memory efficient and typically gives better results (however see my caveats below). It builds on the RF ensemble idea but has true learning. I suggested this to Kate earlier but she ran into errors with it. I could see that given early versions were flakey. But now stable.  Here would be my top choice:

<pre>
from lightgbm import LGBMClassifier
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform
</pre>

###If going to forego tuning, here is my best guess

<pre>
num_classes = 4  #not sure about this

final_model = LGBMClassifier(
    n_estimators=1000,         # Use early stopping to minimize
    learning_rate=0.1,        # Standard value, reliable
    num_leaves=31,           # Default, good balance of complexity/performance
    max_depth=6,             # Prevent overfitting
    min_child_samples=20,    # ~0.02% of data
    subsample=0.8,           # Use 80% of data per tree
    colsample_bytree=0.8,    # Use 80% of features per tree
    reg_alpha=0.1,           # Light L1 regularization
    reg_lambda=0.1,          # Light L2 regularization
    n_jobs=-1,               # Use all cores for single model
    random_state=42,
    objective='multiclass',  # Specifically for n-class problem
    num_class=num_classes    # Specify number of classes
)

final_model.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],   #note will have to do two splitting rounds to get this
    early_stopping_rounds=20,
    eval_metric='multi_logloss'  # for multiclass
)
</pre>

###Below I am introducing tuning, which I did not see in your code

This is not optimized for memory efficiency. If memory becomes an issue, we can change parameter values to reduce memory usage.

<pre>
param_distributions = {
    'learning_rate': uniform(0.01, 0.3),
    'num_leaves': randint(20, 255),
    'max_depth': randint(3, 30),
    'min_child_samples': randint(5, 50),
    'subsample': uniform(0.7, 0.3),
    'colsample_bytree': uniform(0.7, 0.3),
    'reg_alpha': uniform(0, 2),
    'reg_lambda': uniform(0, 2),
}

# Can use RandomizedSearchCV
search = RandomizedSearchCV(
    LGBMClassifier(
        n_estimators=1000,  # Set high for early stopping
        n_jobs=-1
    ),
    param_distributions,
    n_iter=25,   #to save memory at cost of accuracy
    cv=[(slice(None), slice(None))],  # No CV splitting given using validation set
    scoring='accuracy',
    verbose=2,
    random_state=42,
    n_jobs=1  #to save memory at cost of time
)

# Define fit parameters for early stopping
fit_params = {
    'eval_metric': 'multi_logloss',
    'early_stopping_rounds': 20,
    'eval_set': [(X_val, y_val)],   #requires two stage split
    'verbose': False
}

search.fit(X_train, y_train, **fit_params)

# Use best results
final_model = search.best_estimator_
print("Best parameters:", search.best_params_)
print("Best score:", search.best_score_)
print("Optimal number of trees:", best_model.best_iteration_)
</pre>

###Testing can still be done in chunks:
<pre>
results = evaluate_in_chunks(final_model, X_test, y_test)

print("Accuracy:", results['accuracy'])
print("Precision:", results['precision'])
print("Recall:", results['recall'])
print("F1 Score:", results['f1'])
print("\nConfusion Matrix:\n", results['confusion_matrix'])
</pre>

##Reasons not to use boosting

1. Training Speed:
- LightGBM is generally faster than XGBoost but slower than RF for a few reasons:
  - It needs to calculate gradients and update residuals sequentially
  - Each tree depends on previous predictions
  - However, its leaf-wise growth strategy is faster than level-wise tree growth

2. Memory Usage with OOB:
- RF with OOB requires keeping all unused samples for each tree in memory
- LightGBM is typically more memory efficient because:
  - It uses histogram-based splitting
  - Doesn't need to store out-of-bag samples for each tree
  - Uses discretized feature values rather than raw values

You can generally expect:
- Slower training than RF (though faster than XGBoost)
- Lower memory usage, especially with OOB enabled on RF

However:
- The exact speed difference depends on your dataset size and feature count
- Memory usage advantage becomes more pronounced with larger datasets
- If you're using very small datasets, these differences might be negligible

###More generally

Random Forests work well with high variability because:
- Each tree gets a random subset of data and features
- Trees can grow deep to capture complex patterns
- Trees are independent, so they can model different aspects of the variability
- Averaging many diverse trees helps handle noise and outliers

Boosting (like XGBoost/LightGBM) works better with more consistent patterns because:
- It builds trees sequentially, each focusing on correcting previous errors
- High variability can lead to overfitting as later trees may try too hard to fit noise
- The learning rate and sequential nature assume some underlying pattern to gradually improve upon
- Works best when there's a clear signal to learn from

However:
- Both can work well on either type of data with proper tuning
- The distinction becomes more important with limited data
- Modern boosting implementations have features to help handle variability better

# **Apply Prediction**

In [ ]:

slices = int(round(len(img)/20))

test = True

while test == True:
    try:
        class_preds = list()

        temp = rf.predict(img[0:slices+1,:])
        class_preds.append(temp)

        for i in range(slices,len(img),slices):
            print('{} %, derzeit: {}'.format((i*100)/(len(img)), i))
            temp = rf.predict(img[i+1:i+(slices+1),:])
            class_preds.append(temp)

    except MemoryError as error:
        slices = slices/4
        print('Not enought RAM, new slices = {}'.format(slices))

    else:
        test = False
else:
    print('Class prediction was successful without slicing!')
#concatenate all slices and re-shape it to the original extend
try:
    class_prediction = np.concatenate(class_preds,axis = 0)
except NameError:
    print('No slicing was necessary!')

class_prediction = class_prediction.reshape(old_shape[:2])
print('Reshaped back to {}'.format(class_prediction.shape))


# # generate mask image from red band
# mask = np.copy(img[:,:,0])
# mask[mask > 0.0] = 1.0 # all actual pixels have a value of 1.0

# plot mask

# plt.imshow(mask)

# mask classification an plot

class_prediction.astype(np.float16)
class_prediction_ = class_prediction*mask

cols = class_prediction.shape[1]
rows = class_prediction.shape[0]

class_prediction_.astype(np.float16)

driver = gdal.GetDriverByName("gtiff")
outdata = driver.Create(classification_image, cols, rows, 1, gdal.GDT_UInt16)
outdata.SetGeoTransform(img_ds.GetGeoTransform())##sets same geotransform as input
outdata.SetProjection(img_ds.GetProjection())##sets same projection as input
outdata.GetRasterBand(1).WriteArray(class_prediction_)
outdata.FlushCache() ##saves to disk!!
del outdata
print('Image saved to: {}'.format(classification_image))